## Excitation Lattice Model Simulation

In [3]:
import numpy as np
import os
import subprocess

def delete_directory(directory_path):
    command = ['rm', '-rf', directory_path]
    subprocess.run(command)

#Let's submit a hundred jobs
#We want to simulate at different system sizes
#Let's do small and large system sizes
Llist = [30,100]

#Old list of kappa values
#kappalist = [3/8,1/2,2/3]
kappalist = [1/4,3/8,1/2,5/8,2/3]

#We'll play with one lattice size for now
ldlist = [4/3]

#And at different replicas
#I want as many replicas as needed to get statistics of a run with system size L=200. 
copies = 10*200**2/np.array(Llist)**2
gsdcopies = np.copy(copies)
gsdcopies = gsdcopies.astype(int)
gsdcopies[gsdcopies < 1] = 1

#Iterate every system size
for i, L in enumerate(Llist):
    
    #Iterate every interaction strength
    for k, kappa in enumerate(kappalist):
        
        #For temperatures, keep in mind that everything is collapsible in kappa*betaJsigma
        #And the lowest temperature we've done is betaJ = 5.0 and kappa = 1/2 and betaJ = 3.0
        #And so for every kappa, we can keep things simple
        
        kJlist = 0.5*np.array([3.0,4.0,5.0,6.0])
        Jlist = kJlist/kappa
        
        #Iterate every temperature
        for j, J in enumerate(Jlist):
            
            #Iterate every lattice spacing
            for l, ld in enumerate(ldlist):
                
                #Job name and directory
                jobname = 'exc_'+str(L)+'_'+'/{}/{}/{}'.format(j+1,k+1,l+1)
                direc = 'L='+str(L)+'/{}/{}/{}'.format(j+1,k+1,l+1)
                
                #I can create a slurm bash script here
                if not os.path.exists(direc):
                    os.makedirs(direc)
                f = open('{}/script_list.txt'.format(direc),'w')
                
                #Iterate over number of replicas/copies we want to do
                #We want to create a text file full with commands to run the code
                #The SLURM script will then use GNU Parallel to run these commands in parallel
                for m in range(copies[i].astype(int)):
                    direc = 'L='+str(L)+'/{}/{}/{}'.format(j+1,k+1,l+1)
                    x = "python runkmc_restart.py {} {} {} {} {} {} \n".format(L,J,kappa,ld,m,gsdcopies[i])
                    f.write(x)
                f.close()
                
                #Now set up SLURM scripts for different clusters
            
                #(1) Lawrencium
                #Note: We only have two nodes, with around 40 cores to play with
                #This cluster is now what you want to use to run a lot of replicas
                numcpus = copies[i].astype(int)
                if numcpus > 40:
                    numcpus = 40
                f = open('{}/lawrenciumsubmitjob.sh'.format(direc),'w')
                text = """#!/bin/bash
#SBATCH --nodes=1
#SBATCH --job-name={}
#SBATCH --qos=condo_chandra_lr6
#SBATCH --partition=csd_lr6_192
#SBATCH --account=lr_chandra
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task={}
#SBATCH --time=720:00:00
#SBATCH --mail-type=all
#SBATCH --mail-user=muhammad_hasyim@berkeley.edu

eval "$(conda shell.bash hook)"
module load parallel
conda activate env-cupy
export SLURM_CPU_BIND="cores"

#run the application:
parallel --delay 1 -j $SLURM_CPUS_PER_TASK < script_list.txt & 
wait \n""".format(jobname,numcpus)  
                f.write(text)

                numcpus = copies[i].astype(int)
                if numcpus > 24:
                    numcpus = 24

                #(2) Savio
                #Note: We only have four nodes, with around 24 cores to play with
                #But! We can run these jobs for months and that's the nice thing about them. 
                f = open('{}/saviosubmitjob.sh'.format(direc),'w')
                text = """#!/bin/bash
#SBATCH --nodes=1
#SBATCH --job-name={}
#SBATCH --account=co_kranthi
#SBATCH --partition=savio2
#SBATCH --qos=kranthi_savio2_normal
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task={}
#SBATCH --time=8640:00:00
#SBATCH --mail-type=all
#SBATCH --mail-user=muhammad_hasyim@berkeley.edu

eval "$(conda shell.bash hook)"
module load gnu-parallel/2019.03.22
conda activate env-cupy
export SLURM_CPU_BIND="cores"

#run the application:
parallel --delay 1 -j $SLURM_CPUS_PER_TASK < script_list.txt & 
wait \n""".format(jobname,numcpus)  
                f.write(text)
                f.close()